```{index} single: Pyomo; variables 
```
```{index} single: Pyomo; expressions 
```
```{index} single: Pyomo; decorators
```
```{index} single: solver; cbc
```

# A Simple Pyomo Model

Pyomo is an algebraic modeling language embedded within Python for the purpose of implementing mathematical optimization applications. Pyomo is used to create model objects, then add to the model objects representing decision variables, expressions, objective functions, and constraints. When the model is complete, Pyomo methods can be used to solve models with a choice of open-source and commercial solvers. 

This notebook introduces the basic elements of Pyomo for the case study of a small production planning problem. A more complete version of the problem is revisited in Chapter 10 to demonstrate stochastic and robust optimization. This notebook introduces components from the Pyomo library common to most applications:

* [Variables](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Variables.html)
* [Objectives](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Objectives.html)
* [Constraints](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Constraints.html)
* [Expressions](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html)
* [SolverFactory]()

The first version of the model is a direct translation of the mathematics into corresponding Pyomo components using `Var()`, `Objective()`, `Constraint` and `SolverFactory`. No attempt is made at generalizing the model, and parameter values are used directly in expressions. 

This generates a satisfactory solution and would be enough for examples involving a small number of decision variables and constraints. This version of the model provides a tutorial introduction to the use of variables, expressions, objectives, and constraints.

These notebooks use Python decorators to designate Pyomo objectives, constraints, and other model components. This is a new feature available in recent versions of Pyomo.  Decorators may be unfamiliar to new users of Python, but are  worth the effort of learning for a remarkable gain in the readability of Pyomo models.

## Preliminary Step: Install Pyomo and solvers

It is necessary to install the Pyomo library and any solvers that will be used to compute numerical solutions. This cell downloads a Python module that will check if Pyomo and particular solvers have been previously af not, do the necessary installations. r. This cell will work on Google Colab and most laptops. The installation needs to be one-time for a laptop, and done for each new Google Colab session.

In [10]:
import requests
import types

url = "https://raw.githubusercontent.com/mobook/MO-book/main/python/helper.py"
helper = types.ModuleType("helper")
exec(requests.get(url).content, helper.__dict__)

helper.install_pyomo()
helper.install_cbc()

pyomo was previously installed
cbc was previously installed


True

## Step 1. Import Pyomo

The first coding step in creating a Pyomo model is to import the needed components from the Pyomo library into the Python environment. Importing  `pyomo.environ` will provides the most commonly used components for Pyomo model building. 

This collection of notebooks uses a consistent convention of assigning a `pyo` prefix to objects imported from `pyomo.environ`.

In [45]:
import pyomo.environ as pyo

## Step 2. Create a `ConcreteModel` object

A Pyomo model is stored in a Python workspace with a standard Python variable name. The model object can be created with either `pyo.ConcreteModel()` or `pyo.AbstractModel()`. `pyo.ConcreteModel` is used when the problem data is known at the time the model is created, which is the case here. 

The following cell creates an instance of ConcreteModel object and stores it in a Python variable named `model`. The name can be any valid Python variable, but keep it short since the name will be a prefix for every Pyomo variable and constraint. An optional string is used to label subsequent model display.

In [46]:
# create model
model = pyo.ConcreteModel("Production Planning: Version 1")

The `.display()` method is a convenient means of displaying the current contents of a Pyomo model.

In [47]:
#display model
model.display()

Model 'Production Planning: Version 1'

  Variables:
    None

  Objectives:
    None

  Constraints:
    None


## Step 3. Create decision variables

Decision variables are created with the Pyomo `Var()`. The decisions variables can be assigned to any valid Python identifier. Here we assign the decision variables to the model instance using the Python 'dot' notation using a name derived from the mathematical model. 

`Var()` accepts optional keyword arguments:

* `bounds` specifies a tuple containing lower and upper bounds. A good modeling practice is to specify any known and fixed bounds on the decision variables as they are created. If one of the bounds is unknown, use `None` as a placeholder.

* `initialize` specifies initial values for the decision variables. This isn't normally required, but is useful in this tutorial example where we want to display the model as it is being built.

* `domain` keyword specifies the type of decision variable. By default, the domain is all real numbers including negative and positive values. 

The use of the optional keywords is shown in the following cell.

In [48]:
# create decision variables
model.x_M = pyo.Var(bounds=(0, None), initialize=0)
model.x_A = pyo.Var(bounds=(0, 80), initialize=0)
model.x_B = pyo.Var(bounds=(0, 100), initialize=0)

model.y_U = pyo.Var(bounds=(0, 40), initialize=0)
model.y_V = pyo.Var(bounds=(0, None), initialize=0)

# display updated model
model.display()

Model 'Production Planning: Version 1'

  Variables:
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :  None : False : False :  Reals
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :    80 : False : False :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :   100 : False : False :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :    40 : False : False :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :  None : False : False :  Reals

  Objectives:
    None

  Constraints:
    None


## Step 4. Create expressions

The next cell creates linear expressions for revenue and cost and assigns them to the model as `model.revenue` and `model.cost`, respectively. These expressions will be used to create the profit objective. The expressions can be printed to verify correctness.

In [49]:
# create expressions
model.revenue = 270 * model.y_U + 210 * model.y_V
model.cost = 10 * model.x_M + 50 * model.x_A + 40 * model.x_B

# expressions can be printed
print(model.revenue)
print(model.cost)

270*y_U + 210*y_V
10*x_M + 50*x_A + 40*x_B


## Step 5. Create objective

The objective is to maximize profit, where  profit is defined as the difference between revenue and cost.

The expression to be optimized is defined using the `expr` keyword argument. The type of optimization (minimization or maximization) can be specified using the `sense` keyword argument. For example, to maximize profit, defined as the difference between revenue and cost: 

    model.profit = pyo.Objective(expr = model.revenue - model.cost, sense = pyo.maximize)
    
Recent releases of Pyomo also provide a decorator syntax to create an objective function. Using the @model.Objective(sense = pyo.maximize) decorator:

    @model.Objective(sense = pyo.maximize)
    def profit(model):
        return model.revenue - model.cost

Decorators change the behavior of a function. In this case, the decorator `model.Objective()` modifies `profit()` to return an expression for the profit. The  keyword `sense` sets the type  objective (`pyo.maximize` or `pyo.minimize`). The decorated function profit() takes the Pyomo model as its first argument and its name is added to the model attributes.

This alternative syntax allows for improved readability and maintainability of more complex models.  Decorators provide a way to simplify the syntax for creating different objects, such as objectives, expressions, constraints, etc., in optimization applications.  

In [50]:
@model.Objective(sense=pyo.maximize)
def profit(model):
    return model.revenue - model.cost

model.display()

Model 'Production Planning: Version 1'

  Variables:
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :  None : False : False :  Reals
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :    80 : False : False :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :   100 : False : False :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :    40 : False : False :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :  None : False : False :  Reals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :     0

  Constraints:
    None


## Step 6. Add constraints

Constraints are logical relationships between expressions that define the range of feasible solutions in an optimization problem. The logical relationships can be equality (`==`), less-than (`<=`), or greater-than (`>=`). 

`pyo.Constraint()` is a Pyomo class to creating constraints between expressions. A constraint consists of two expressions separated by one of the logical relationships. The constraint is passed as a keyword argument `expr` to `pyo.Constraint()`. For this application the constraints can be expressed as  

    model.raw_materials = pyo.Constraint(expr = 10 * model.y_U + 9 * model.y_V <= model.x_M)
    model.labor_A = pyo.Constraint(expr = 2 * model.y_U + 1 * model.y_V <= model.x_A)
    model.labor_B = pyo.Constraint(expr = 1 * model.y_U + 1 * model.y_V <= model.x_B)
    
A `model.Constraint()` decorator is an alternative syntax. The decorator 'tags' the output of the following function as a constraint. For the present example the decorator system is would be

In [51]:
@model.Constraint()
def raw_materials(model):
    return 10 * model.y_U + 9 * model.y_V <= model.x_M

@model.Constraint()
def labor_A(model):
    return 2 * model.y_U + 1 * model.y_V <= model.x_A

@model.Constraint()
def labor_B(model):
    return 1 * model.y_U + 1 * model.y_V <= model.x_B

model.pprint()

5 Var Declarations
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :    80 : False : False :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :   100 : False : False :  Reals
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :  None : False : False :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :    40 : False : False :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     0 :  None : False : False :  Reals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 270*y_U + 210*y_V - (10*x_M + 50*x_A + 40*x_B)

3 Constraint Dec

## Step 7. Solve the model

With the model now fully specified, the next step is to compute a solution. A solver object is created with `SolverFactory`, then applied to the model as shown in the following cell. Here we have chosen to use the open source [COIN-OR Cbc](https://github.com/coin-or/Cbc)  ("COIN-OR branch and cut") solver for mixed integer linear programming. There are suitable solver, such as the open source GNU Linear Programming Kit [GLPK](https://en.wikibooks.org/wiki/GLPK) and commercial solvers such as CPLEX, Gurobi, and Mosek.

This cell shows how to use create and use a solver. The optional keyword `tee=True` causes the solver to print its output to the output. This can be useful for verifying the model is bdebugging problems

In [70]:
solver = pyo.SolverFactory("cbc")
results = solver.solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Apr 19 2021 

command line - /Users/jeff/opt/anaconda3/bin/cbc -printingOptions all -import /var/folders/cm/z3t28j296f98jdp1vqyplkz00000gn/T/tmpb1fiin9x.pyomo.lp -stat=1 -solve -solu /var/folders/cm/z3t28j296f98jdp1vqyplkz00000gn/T/tmpb1fiin9x.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 3 (-1) rows, 5 (-1) columns and 9 (-1) elements
Statistics for presolved model


Problem has 3 rows, 5 columns (5 with objective) and 9 elements
There are 3 singletons with objective 
Column breakdown:
2 of type 0.0->inf, 3 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
2 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 

The solver returns a results object that reports inforomation about the problem, and theti

## Step 8. Reporting the solution

Pyomo provides methods to display the solution to an optimization problem. The ba

For an overview of other ways to report and visualize the solutions, see also the appendix of [this notebook](../04/gasoline-distribution.ipynb).

In [53]:
model.__dict__

{'_ctypes': {pyomo.core.base.var.Var: [0, 4, 5],
  pyomo.core.base.objective.Objective: [5, 5, 1],
  pyomo.core.base.constraint.Constraint: [6, 8, 3]},
 '_decl': {'x_M': 0,
  'x_A': 1,
  'x_B': 2,
  'y_U': 3,
  'y_V': 4,
  'profit': 5,
  'raw_materials': 6,
  'labor_A': 7,
  'labor_B': 8},
 '_decl_order': [(<pyomo.core.base.var.ScalarVar at 0x112f58e40>, 1),
  (<pyomo.core.base.var.ScalarVar at 0x112f58d60>, 2),
  (<pyomo.core.base.var.ScalarVar at 0x112f589e0>, 3),
  (<pyomo.core.base.var.ScalarVar at 0x112f58510>, 4),
  (<pyomo.core.base.var.ScalarVar at 0x112f58740>, None),
  (<pyomo.core.base.objective.ScalarObjective at 0x112b34ca0>, None),
  (<pyomo.core.base.constraint.ScalarConstraint at 0x112f582e0>, 7),
  (<pyomo.core.base.constraint.ScalarConstraint at 0x112f58820>, 8),
  (<pyomo.core.base.constraint.ScalarConstraint at 0x112f58b30>, None)],
 '_suppress_ctypes': set(),
 '_dense': True,
 '_ctype': pyomo.core.base.block.Block,
 'doc': None,
 '_name': 'Production Planning: Vers

In [59]:
pyo.value(model.x_M)

720.0

In [60]:
pyo.value(model.labor_A)

0.0

In [61]:
pyo.value(model.profit)

2400.0

In [62]:
model.pprint()

5 Var Declarations
    x_A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  80.0 :    80 : False : False :  Reals
    x_B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  80.0 :   100 : False : False :  Reals
    x_M : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 720.0 :  None : False : False :  Reals
    y_U : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :    40 : False : False :  Reals
    y_V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  80.0 :  None : False : False :  Reals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 270*y_U + 210*y_V - (10*x_M + 50*x_A + 40*x_B)

3 Constraint Dec